# Hydraulic Systems - Data Exploration & Preparation

## Kontext

Die Daten enthalten **Zeitreihen-Messungen** von Sensoren:
- **Zeilen** = Zyklen (2205 Messzyklen)
- **Spalten** = Zeitpunkte innerhalb eines 60-Sekunden-Zyklus

### Sensor-Sampling-Raten:
- **100 Hz** (6000 Messpunkte): PS1-PS6, EPS1
- **10 Hz** (600 Messpunkte): FS1, FS2  
- **1 Hz** (60 Messpunkte): TS1-TS4, VS1, CE, CP, SE

### Strategie:
1. **Rohdaten laden** und Struktur verstehen
2. **Feature Engineering**: Aggregationen pro Zyklus (mean, std, min, max, etc.)
3. **Zielvariablen** aus profile.txt einbinden
4. **Bereinigte Features** exportieren

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Plotting-Style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✓ Imports erfolgreich")

## 1. Datenstruktur verstehen

Schauen wir uns exemplarisch eine Datei an:

In [ ]:
# Beispiel: TS1 (Temperature Sensor 1, 1 Hz = 60 Messpunkte)
ts1 = pd.read_csv('../data/TS1.txt', sep=r'\s+', header=None, engine='python')

print(f"TS1 Shape: {ts1.shape}")
print(f"  → {ts1.shape[0]} Zyklen")
print(f"  → {ts1.shape[1]} Zeitpunkte pro Zyklus (1 Hz × 60 Sekunden)\n")

print("Erste 5 Zyklen, erste 10 Zeitpunkte:")
ts1.iloc[:5, :10]

In [ ]:
# Beispiel: PS1 (Pressure Sensor 1, 100 Hz = 6000 Messpunkte)
ps1 = pd.read_csv('../data/PS1.txt', sep=r'\s+', header=None, engine='python')

print(f"PS1 Shape: {ps1.shape}")
print(f"  → {ps1.shape[0]} Zyklen")
print(f"  → {ps1.shape[1]} Zeitpunkte pro Zyklus (100 Hz × 60 Sekunden)\n")

print("Erste 3 Zyklen, erste 10 Zeitpunkte:")
ps1.iloc[:3, :10]

## 2. Visualisierung: Ein einzelner Zyklus

Plotten wir einen kompletten Zyklus für TS1:

In [ ]:
# Zyklus #100 von TS1
cycle_idx = 100
cycle_data = ts1.iloc[cycle_idx, :].values

plt.figure(figsize=(14, 4))
plt.plot(cycle_data, marker='.', linestyle='-', linewidth=0.8)
plt.title(f'TS1 - Zyklus #{cycle_idx} (60 Sekunden @ 1 Hz)', fontsize=14)
plt.xlabel('Zeitpunkt (0-59 Sekunden)')
plt.ylabel('Temperatur (°C)')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Min: {cycle_data.min():.2f} °C")
print(f"Max: {cycle_data.max():.2f} °C")
print(f"Mean: {cycle_data.mean():.2f} °C")
print(f"Std: {cycle_data.std():.2f} °C")

## 3. Feature Engineering: Aggregationen

Statt 43.680 Zeitpunkte zu nutzen, erstellen wir **aussagekräftige Features** pro Sensor/Zyklus:

- `mean`: Durchschnitt
- `std`: Standardabweichung (Variabilität)
- `min`: Minimum
- `max`: Maximum
- `median`: Median
- `q25`, `q75`: Quartile
- `range`: max - min

In [ ]:
def extract_features(df, sensor_name):
    """
    Extrahiert aggregierte Features aus Zeitreihen-DataFrame.
    
    Args:
        df: DataFrame mit Zyklen × Zeitpunkten
        sensor_name: Name des Sensors (z.B. 'ts1')
    
    Returns:
        DataFrame mit aggregierten Features pro Zyklus
    """
    features = pd.DataFrame()
    
    # Konvertiere zu numerisch (falls nötig)
    df_numeric = df.apply(pd.to_numeric, errors='coerce')
    
    features[f'{sensor_name}_mean'] = df_numeric.mean(axis=1)
    features[f'{sensor_name}_std'] = df_numeric.std(axis=1)
    features[f'{sensor_name}_min'] = df_numeric.min(axis=1)
    features[f'{sensor_name}_max'] = df_numeric.max(axis=1)
    features[f'{sensor_name}_median'] = df_numeric.median(axis=1)
    features[f'{sensor_name}_q25'] = df_numeric.quantile(0.25, axis=1)
    features[f'{sensor_name}_q75'] = df_numeric.quantile(0.75, axis=1)
    features[f'{sensor_name}_range'] = features[f'{sensor_name}_max'] - features[f'{sensor_name}_min']
    
    return features

# Test mit TS1
ts1_features = extract_features(ts1, 'ts1')
print(f"TS1 Features Shape: {ts1_features.shape}")
print(f"\nSpalten: {list(ts1_features.columns)}")
print("\nErste 5 Zeilen:")
ts1_features.head()

## 4. Alle Sensoren verarbeiten

Jetzt laden und aggregieren wir **alle** Sensor-Dateien:

In [ ]:
def load_and_aggregate_all(data_path='../data'):
    """
    Lädt alle Sensor-Dateien und extrahiert aggregierte Features.
    """
    data_dir = Path(data_path)
    all_features = []
    
    sensor_files = sorted(data_dir.glob('*.txt'))
    
    for file_path in sensor_files:
        sensor_name = file_path.stem.lower()
        
        print(f"Verarbeite {sensor_name}...", end=' ')
        
        # Lade Daten
        df = pd.read_csv(file_path, sep=r'\s+', header=None, engine='python')
        
        # Extrahiere Features
        features = extract_features(df, sensor_name)
        all_features.append(features)
        
        print(f"✓ ({df.shape[0]} Zyklen × {df.shape[1]} Zeitpunkte → {features.shape[1]} Features)")
    
    # Alle Features zusammenführen
    combined = pd.concat(all_features, axis=1)
    
    return combined

# Führe aus
features_df = load_and_aggregate_all()

print(f"\n{'='*70}")
print(f"Final Features Shape: {features_df.shape}")
print(f"  → {features_df.shape[0]} Zyklen")
print(f"  → {features_df.shape[1]} aggregierte Features")
print(f"{'='*70}")

In [ ]:
# Übersicht über Features
print("Spaltenübersicht:")
print(features_df.columns.tolist())

In [ ]:
# Erste Zeilen
features_df.head(10)

## 5. Zielvariablen laden (profile.txt)

Laut Dokumentation enthält `profile.txt` die Ziel-Labels für jeden Zyklus.

In [ ]:
# Lade profile.txt
profile = pd.read_csv('../docs/profile.txt', sep='\t', header=None)

print(f"Profile Shape: {profile.shape}")
print(f"\nErste 10 Zeilen:")
print(profile.head(10))

print(f"\nUnique Werte pro Spalte:")
for i in range(profile.shape[1]):
    print(f"  Spalte {i}: {sorted(profile[i].unique())}")

In [ ]:
# Laut Dokumentation:
# Spalte 0: Cooler condition (3, 20, 100)
# Spalte 1: Valve condition
# Spalte 2: Pump leakage
# Spalte 3: Accumulator pressure
# Spalte 4: Stable flag

profile.columns = ['cooler_condition', 'valve_condition', 'pump_leakage', 
                   'accumulator_pressure', 'stable_flag']

print("Profile mit Spaltennamen:")
profile.head(10)

## 6. Features + Targets zusammenführen

In [ ]:
# Prüfe ob Längen übereinstimmen
print(f"Features: {len(features_df)} Zeilen")
print(f"Profile:  {len(profile)} Zeilen")

# Füge zusammen
df_complete = pd.concat([features_df.reset_index(drop=True), 
                         profile.reset_index(drop=True)], axis=1)

print(f"\nKompletter Datensatz: {df_complete.shape}")
df_complete.head()

## 7. Basis-Statistiken

In [ ]:
# Statistiken nur für Features (nicht Targets)
feature_cols = [col for col in df_complete.columns if col not in profile.columns]

stats = df_complete[feature_cols].describe().T
stats['missing'] = df_complete[feature_cols].isna().sum()
stats['missing_pct'] = 100 * stats['missing'] / len(df_complete)

print("Feature-Statistiken:")
stats[['count', 'missing', 'missing_pct', 'mean', 'std', 'min', 'max']]

In [ ]:
# Speichere Statistiken
stats.to_csv('../out/feature_stats.csv')
print("✓ Statistiken gespeichert: out/feature_stats.csv")

## 8. Missing Values prüfen

In [ ]:
missing_summary = pd.DataFrame({
    'column': df_complete.columns,
    'n_missing': df_complete.isna().sum().values,
    'pct_missing': 100 * df_complete.isna().sum().values / len(df_complete)
})

missing_summary = missing_summary[missing_summary['n_missing'] > 0].sort_values('n_missing', ascending=False)

if len(missing_summary) > 0:
    print("Spalten mit Missing Values:")
    print(missing_summary)
else:
    print("✓ Keine Missing Values gefunden!")

## 9. Korrelationsanalyse

Jetzt mit vernünftiger Anzahl an Features (statt 43k):

In [ ]:
# Korrelation nur für Features
corr = df_complete[feature_cols].corr()

print(f"Korrelationsmatrix: {corr.shape}")

# Speichern
corr.to_csv('../out/correlation.csv')
print("✓ Korrelation gespeichert: out/correlation.csv")

In [ ]:
# Heatmap (alle Features)
plt.figure(figsize=(20, 18))
sns.heatmap(corr, cmap='coolwarm', center=0, square=True, 
            linewidths=0.1, cbar_kws={"shrink": 0.8})
plt.title('Korrelations-Heatmap (Alle Features)', fontsize=16, pad=20)
plt.tight_layout()
plt.savefig('../out/correlation_heatmap_full.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Heatmap gespeichert: out/correlation_heatmap_full.png")

## 10. Top-Korrelationen finden

In [ ]:
# Finde höchste Korrelationen (ohne Diagonale)
corr_pairs = []

for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        corr_pairs.append({
            'feature1': corr.columns[i],
            'feature2': corr.columns[j],
            'correlation': corr.iloc[i, j]
        })

corr_pairs_df = pd.DataFrame(corr_pairs)
corr_pairs_df = corr_pairs_df.sort_values('correlation', ascending=False, key=abs)

print("Top 20 höchste Korrelationen:")
corr_pairs_df.head(20)

## 11. Visualisierungen: Feature-Verteilungen

In [ ]:
# Beispiel: Mean-Features für verschiedene Sensoren
mean_features = [col for col in feature_cols if '_mean' in col]

fig, axes = plt.subplots(4, 4, figsize=(16, 12))
axes = axes.flatten()

for i, col in enumerate(mean_features[:16]):
    axes[i].hist(df_complete[col].dropna(), bins=50, edgecolor='black', alpha=0.7)
    axes[i].set_title(col, fontsize=10)
    axes[i].set_xlabel('Wert')
    axes[i].set_ylabel('Häufigkeit')
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../out/feature_distributions_mean.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Verteilungen gespeichert: out/feature_distributions_mean.png")

## 12. Boxplots: Features nach Cooler Condition

In [ ]:
# Beispiel: TS1 Mean nach Cooler Condition
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

features_to_plot = ['ts1_mean', 'ts2_mean', 'ps1_mean', 'ps2_mean']

for i, feat in enumerate(features_to_plot):
    if feat in df_complete.columns:
        df_complete.boxplot(column=feat, by='cooler_condition', ax=axes[i])
        axes[i].set_title(f'{feat} by Cooler Condition')
        axes[i].set_xlabel('Cooler Condition')
        axes[i].set_ylabel(feat)

plt.suptitle('')  # Entferne automatischen Titel
plt.tight_layout()
plt.savefig('../out/boxplots_by_cooler.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Boxplots gespeichert: out/boxplots_by_cooler.png")

## 13. Export: Bereinigter Datensatz

In [ ]:
# Exportiere als Parquet (kompakt & schnell)
df_complete.to_parquet('../out/features_complete.parquet', index=False)
print(f"✓ Gespeichert: out/features_complete.parquet ({df_complete.shape})")

# Auch als CSV (für einfache Ansicht)
df_complete.to_csv('../out/features_complete.csv', index=False)
print(f"✓ Gespeichert: out/features_complete.csv")

## 14. Zusammenfassung

### Ergebnisse:

In [ ]:
print("="*70)
print("ZUSAMMENFASSUNG")
print("="*70)
print(f"\nDatenpunkte:")
print(f"  • {df_complete.shape[0]:,} Zyklen")
print(f"  • {len(feature_cols):,} aggregierte Features")
print(f"  • {len(profile.columns)} Zielvariablen")
print(f"\nExportierte Dateien:")
print(f"  • out/features_complete.parquet")
print(f"  • out/features_complete.csv")
print(f"  • out/feature_stats.csv")
print(f"  • out/correlation.csv")
print(f"  • out/correlation_heatmap_full.png")
print(f"  • out/feature_distributions_mean.png")
print(f"  • out/boxplots_by_cooler.png")
print(f"\nZielvariablen:")
for col in profile.columns:
    unique_vals = df_complete[col].unique()
    print(f"  • {col}: {unique_vals}")
print("\n" + "="*70)